The plugin-oriented nature of psiexperiment makes a direct comparision with RefTarScript a bit difficult. However, we generally have the following tasks that need to be performed:

* **Configure the hardware**. This includes defining which channels are connected to which inputs/outputs. 
* **Hardware abstraction layer**. The software should not care whether your speaker is connected to `'Dev32143/ao3'` or `'Dev1/ao1'`. The software just wants a channel named `'speaker'` that it can upload sound waveforms to. The hardware abstraction layer will handle this internally. In psiexperiment, the abstraction layer is provided by `psi.controller.engine`.
* **Define a sequence of parameters to iterate through**. If you're doing a psychophysical task, you'll want to define a sequence of values to test. For example, if you're doing tone in noise detection, you may want to vary the signal-to-noise ratio.
* **Generate and upload a stimulus waveform**. We do auditory experiments. We play sounds and measure the response to sound.
* **Perform actions based on what happens during the experiment**. If the animal licks the spout, what should the program do?


## Hardware configuration

Let's look at an <a href="https://github.com/bburan/psiexperiment/blob/master/psi/application/io/bobcat.enaml#L27">example</a> of a configuration for the computer that runs the gerbil behavior. Note the following:

* There are three outputs (a `PelletDispenser`, `Trigger` and `Toggle`). These are named outputs that participate in the hardware abstraction layer. They are handled by the controller plugin.

* There are three input channels. Two of these (`'nose_poke'` and `'reward_contact'`) are passed through a processing hierarchy that eventually culminates in edge detection.

* All inputs and outputs are named. We can refer to them by name in the program.

```
room_light = controller.get_output('room_light_toggle')
room_light.set_high() # turn on the room light
room_light.set_low()  # turn off the room light
```

## Hardware abstraction

In this particular case, one could configure the room light such that it was controlled by an Arduino.

```
ArduinoEngine:
    DigitalOutput:
        name = 'room_light'
        channel = 'Pin1'
```

The engine provides the abstraction layer. In particular, the engine is responsible for:

* Configuring the input and output channels.
* Responding to requests (e.g., uploading waveforms to analog output channels and toggling the state of digital output channels)
* Continuously polling input channels and passing this through the input processing pipeline. All data acquisition is continuous (if you want epoch-based acquisition, you would add a special input called, `ExtractEpochs` to your input hierarchy).

You can have multiple engines in a single experiment. For example, an Arduino to control the room light and food dispenser and a NIDAQmx to manage the speaker, nose-poke and reward contact sensors.

## Defining parameters and sequences

You can <a href="https://github.com/bburan/psiexperiment/blob/master/psi/controller/appetitive_manifest.enaml#L95">define a list of parameters</a> that control the experiment as well as a <a href="https://github.com/bburan/psiexperiment/blob/master/psi/controller/appetitive_manifest.enaml#L211">set of selectors</a> that allow you to specify a sequence of values to test.

Once you have defined these parameters and sequences, you can iterate through them.

```
context = workbench.get_plugin('psi.context')

context.next_setting(selector='go')
values = context.get_values()

context.next_setting(selector='nogo')
values = context.get_values()
```

This strategy is used in the <a href="https://github.com/bburan/psiexperiment/blob/master/psi/controller/appetitive_plugin.py#L226">appetitive go-nogo experiment</a>.

## Generating waveforms

We've already discussed this briefly. However, what we have not discussed is *how* we tell the program to present a particular waveform. In general, the Engines are *always* polling every output (continuous, epoch, queued, etc.) for samples. If an output is not active (e.g., we are in the intertrial period and therefore there is no target to present), the epoch output will simply return a string of zeros. The engine actually buffers about 10 to 30 seconds of data in the output queues (to protect the experiment from crashing when you stall the computer by checking your email).

We can activate an output called `target` to begin at 10.5 seconds (re. acquisition start):

```
core = workbench.get_plugin('enaml.workbench.core')
core.invoke_command('target.prepare')
core.invoke_command('target.start', {'ts': 10.5})
```

Commands are basically strings that map to curried functions (one of Ivar's favorite types of functions).